In [1]:
import os
import numpy as np
from numpy import array
import matplotlib
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt
from getdist import plots
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# from mockfactory import Catalog
from cosmoprimo.fiducial import DESI
from desilike.theories.galaxy_clustering import FOLPSTracerPowerSpectrumMultipoles, FOLPSRCTracerPowerSpectrumMultipoles,FOLPSRCMoTracerPowerSpectrumMultipoles
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.emulators import EmulatedCalculator, Emulator, TaylorEmulatorEngine
from desilike.likelihoods import ObservablesGaussianLikelihood
from desilike.samplers.emcee import EmceeSampler
from desilike.samples import Profiles, plotting, Chain
from desilike import setup_logging
setup_logging()  # for logging messages


# the cosmology parameters

kmin     = 0.008
kmax     = 0.20
binning  = 0.006
k_ev     = np.arange(kmin, kmax+0.001, binning)
klim     = {ell*2: (kmin,kmax,binning) for ell in range(2)}

redshift    = 1.0
catalogue   = 'fiducial'  # fiducial, Mnu_p, Mnu_ppp -- QUIJOTE catalogue
cosmology   = 'fiducial'
model       = 'FOLPSRCmo'
CovRsf      = 100  # -- covariance rescale factor
r_pks       = ['RSD','1%CATAS','5%CATAS']

# for model in ['FOLPS','FOLPSRC']:

for r_pk in r_pks:
    chain_fn = f'./sampler_result/{cosmology}/chain_FOLPSRCv2_{r_pk}_V{CovRsf}_z{redshift}.npy'
    emulator_fn = f'./emulators/emulator_{cosmology}_z{redshift}.npy'
    print(chain_fn)
    if not os.path.exists(chain_fn):
        filename = []
        filedir = f'/Users/alain/Desktop/projectNU/main/data/kbin2/{catalogue}/{r_pk}_z{redshift}/npy/'
        for file in os.listdir(filedir):
            filename.append(filedir+file)
        covariance = filedir+'*'
        if model == 'FOLPS':
            theory = FOLPSTracerPowerSpectrumMultipoles(pt=EmulatedCalculator.load(emulator_fn))
        if model == 'FOLPSRC':
            theory = FOLPSRCTracerPowerSpectrumMultipoles(pt=EmulatedCalculator.load(emulator_fn))
        if model == 'FOLPSRC_fix':
            theory = FOLPSRCTracerPowerSpectrumMultipoles(pt=EmulatedCalculator.load(emulator_fn))  
        if model == 'FOLPSRCmo':
            theory = FOLPSRCMoTracerPowerSpectrumMultipoles(pt=EmulatedCalculator.load(emulator_fn))      
        observable = TracerPowerSpectrumMultipolesObservable(data= filename,
                                                            covariance= covariance,
                                                            klim=klim,
                                                            theory=theory,
                                                            kin=np.arange(0.001,0.35,0.002))
        likelihood = ObservablesGaussianLikelihood(observable, scale_covariance = 1/CovRsf) #
        if model == 'FOLPSRC_fix':
            likelihood.varied_params['fc'].update(value=0.01,fixed=True)
        likelihood()
        sampler = EmceeSampler(likelihood, seed=42, nwalkers=120, save_fn =chain_fn)
        sampler.run(check={'max_eigen_gr': 0.05}, max_iterations = 9001) # save every 300 iterations

[tsf-484-wpa-2-023.epfl.ch:39032] shmem: mmap: an error occurred while determining whether or not /var/folders/n0/gs1g7p590qg7trf8pp8w29ch0000gn/T//ompi.tsf-484-wpa-2-023.501/jf.0/608960512/sm_segment.tsf-484-wpa-2-023.501.244c0000.0 could be created.


./sampler_result/fiducial/chain_FOLPSRCv2_RSD_V100_z1.0.npy
[000000.01] [0/1] 10-04 17:23  Emulator                  INFO     Loading ./emulators/emulator_fiducial_z1.0.npy.
[000000.57] [0/1] 10-04 17:23  TracerPowerSpectrumMultipolesObservable INFO     Loading 100 files [/Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_122_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_101_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_189_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_141_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_180_z1.0.npy].
[000000.71] [0/1] 10-04 17:23  TracerPowerSpectrumMultipolesObservable INFO     Loading 100 files [/Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_100_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fid